In [1]:
from enchant.utils import levenshtein
import pandas as pd
import numpy as np

In [2]:
cps_df = pd.read_csv('clean_data/cellphones.csv')
tgdd_df = pd.read_csv('clean_data/tgdd.csv')
tiki_df = pd.read_csv('clean_data/tiki.csv')
vt_df = pd.read_csv('clean_data/viettel.csv')

In [3]:
cps_column = []
tgdd_column = []
tiki_column = []
vt_column = []

for e in cps_df.columns:
    cps_column.append(e)

for e in tgdd_df.columns:
    tgdd_column.append(e)

for e in tiki_df.columns:
    tiki_column.append(e)

for e in vt_df.columns:
    vt_column.append(e)

In [11]:
cps_df.set_index('id')

KeyError: "None of ['id'] are in the columns"

In [4]:
def name_matcher(column1, column2):
    res = []
    for c1 in column1:
        tmp = []
        for c2 in column2:
            tmp.append(levenshtein(c1, c2))
        res.append(tmp)
    return res

def jaccard(A, B):
    nominator = A.symmetric_difference(B)
    denominator = A.union(B)
    distance = len(nominator)/len(denominator)    
    return distance

def instance_matcher(df1, df2, column1, column2):
    res = []
    for c1 in column1:
        tmp = []
        for c2 in column2:
            tmp.append(jaccard(set(df1[c1]), set(df2[c2])))
        res.append(tmp)
    return res

# Matching CellphoneS với TGDD

In [5]:
cps_tgdd_name = name_matcher(cps_column, tgdd_column)
max_value = np.array(cps_tgdd_name).max()
cps_tgdd_name = np.array(cps_tgdd_name)/max_value
# cps_tgdd_name = pd.DataFrame(cps_tgdd_name, columns = [tgdd_column], index=[cps_column])
# cps_tgdd_name

cps_tgdd_instance = instance_matcher(cps_df, tgdd_df, cps_column, tgdd_column)
cps_tgdd_instance = np.array(cps_tgdd_instance)
# cps_tgdd_instance = pd.DataFrame(cps_tgdd_instance, columns = [tgdd_column], index=[cps_column])
# cps_tgdd_instance

cps_tgdd = np.add(cps_tgdd_name, cps_tgdd_instance)/2
cps_tgdd = pd.DataFrame(cps_tgdd, columns = [tgdd_column], index=[cps_column])
cps_tgdd

,ten,gia,cong_nghe_man_hinh,cpu,ram,bo_nho_trong,sim,pin,kich_thuoc_man_hinh
ten,0.485130,0.583333,0.944444,0.583333,0.583333,0.777778,0.583333,0.555556,0.972222
gia,0.583333,0.437853,0.944444,0.583333,0.583333,0.833333,0.555556,0.555556,0.972222
kich_thuoc_man_hinh,0.972222,0.972222,0.750000,0.972222,1.000000,0.888889,0.972222,0.972222,0.337209
cong_nghe_man_hinh,0.944444,0.944444,0.181818,0.956093,0.972222,0.833333,0.972222,0.944444,0.750000
cpu,0.583333,0.583333,0.972222,0.473333,0.583333,0.833333,0.583333,0.583333,0.972222
ram,0.583333,0.583333,0.972222,0.583333,0.111111,0.767094,0.555556,0.583333,1.000000
bo_nho_trong,0.777778,0.833333,0.833333,0.833333,0.738889,0.250000,0.833333,0.805556,0.888889
pin,0.555556,0.555556,0.944444,0.579812,0.583333,0.805556,0.555556,0.488372,0.972222
sim,0.583333,0.555556,0.972222,0.564815,0.555556,0.833333,0.500000,0.555556,0.972222


In [6]:
cps_tgdd = cps_tgdd.idxmin(axis=1)
cps_tgdd

ten                                    (ten,)
gia                                    (gia,)
kich_thuoc_man_hinh    (kich_thuoc_man_hinh,)
cong_nghe_man_hinh      (cong_nghe_man_hinh,)
cpu                                    (cpu,)
ram                                    (ram,)
bo_nho_trong                  (bo_nho_trong,)
pin                                    (pin,)
sim                                    (sim,)
dtype: object

# Matching CellphoneS với Tiki

In [7]:
cps_tiki_name = name_matcher(cps_column, tiki_column)
max_value = np.array(cps_tiki_name).max()
cps_tiki_name = np.array(cps_tiki_name)/max_value
# cps_tgdd_name = pd.DataFrame(cps_tgdd_name, columns = [tgdd_column], index=[cps_column])
# cps_tgdd_name

cps_tiki_instance = instance_matcher(cps_df, tiki_df, cps_column, tiki_column)
cps_tiki_instance = np.array(cps_tiki_instance)
# cps_tgdd_instance = pd.DataFrame(cps_tgdd_instance, columns = [tgdd_column], index=[cps_column])
# cps_tgdd_instance

cps_tiki = np.add(cps_tiki_name, cps_tiki_instance)/2
cps_tiki = pd.DataFrame(cps_tiki, columns = [tiki_column], index=[cps_column])
cps_tiki

,ten,gia,cpu,cong_nghe_man_hinh,pin,sim,ram,bo_nho_trong,kich_thuoc_man_hinh
ten,0.495475,0.583333,0.583333,0.944444,0.555556,0.583333,0.583333,0.777778,0.972222
gia,0.583333,0.433225,0.583333,0.944444,0.555556,0.555556,0.583333,0.833333,0.972222
kich_thuoc_man_hinh,0.972222,0.972222,0.972222,0.750000,0.972222,0.972222,1.000000,0.888889,0.272727
cong_nghe_man_hinh,0.944444,0.944444,0.966846,0.379310,0.929293,0.957516,0.972222,0.833333,0.750000
cpu,0.583333,0.583333,0.441624,0.969117,0.580189,0.580208,0.583333,0.833333,0.972222
ram,0.583333,0.583333,0.583333,0.972222,0.583333,0.555556,0.208333,0.591270,0.967742
bo_nho_trong,0.777778,0.833333,0.833333,0.833333,0.805556,0.833333,0.638889,0.250000,0.888889
pin,0.555556,0.555556,0.580882,0.941020,0.478417,0.552107,0.583333,0.805556,0.972222
sim,0.583333,0.555556,0.577715,0.956093,0.538314,0.465517,0.555556,0.833333,0.972222


In [8]:
cps_tiki = cps_tiki.idxmin(axis=1)
cps_tiki

ten                                    (ten,)
gia                                    (gia,)
kich_thuoc_man_hinh    (kich_thuoc_man_hinh,)
cong_nghe_man_hinh      (cong_nghe_man_hinh,)
cpu                                    (cpu,)
ram                                    (ram,)
bo_nho_trong                  (bo_nho_trong,)
pin                                    (pin,)
sim                                    (sim,)
dtype: object

# Matching CellphoneS với Tiki

In [9]:
cps_vt_name = name_matcher(cps_column, vt_column)
max_value = np.array(cps_vt_name).max()
cps_vt_name = np.array(cps_vt_name)/max_value
# cps_tgdd_name = pd.DataFrame(cps_tgdd_name, columns = [tgdd_column], index=[cps_column])
# cps_tgdd_name

cps_vt_instance = instance_matcher(cps_df, vt_df, cps_column, vt_column)
cps_vt_instance = np.array(cps_vt_instance)
# cps_tgdd_instance = pd.DataFrame(cps_tgdd_instance, columns = [tgdd_column], index=[cps_column])
# cps_tgdd_instance

cps_vt = np.add(cps_vt_name, cps_vt_instance)/2
cps_vt = pd.DataFrame(cps_vt, columns = [vt_column], index=[cps_column])
cps_vt

,ten,gia,cpu,ram,bo_nho_trong,pin,sim,kich_thuoc_man_hinh,cong_nghe_man_hinh
ten,0.442368,0.583333,0.583333,0.583333,0.777778,0.555556,0.583333,0.972222,0.944444
gia,0.583333,0.406699,0.583333,0.583333,0.833333,0.555556,0.555556,0.972222,0.944444
kich_thuoc_man_hinh,0.972222,0.972222,0.972222,1.000000,0.888889,0.972222,0.972222,0.255319,0.750000
cong_nghe_man_hinh,0.944444,0.944444,0.966198,0.972222,0.833333,0.944444,0.964026,0.750000,0.340909
cpu,0.583333,0.583333,0.480198,0.583333,0.833333,0.583333,0.580660,0.972222,0.968975
ram,0.583333,0.583333,0.583333,0.111111,0.781746,0.583333,0.555556,0.958333,0.972222
bo_nho_trong,0.777778,0.833333,0.833333,0.738889,0.233333,0.805556,0.833333,0.876068,0.833333
pin,0.555556,0.555556,0.580756,0.583333,0.805556,0.490196,0.552649,0.972222,0.940847
sim,0.583333,0.555556,0.577004,0.555556,0.833333,0.555556,0.491228,0.972222,0.951389


In [10]:
cps_vt = cps_vt.idxmin(axis=1)
cps_vt

ten                                    (ten,)
gia                                    (gia,)
kich_thuoc_man_hinh    (kich_thuoc_man_hinh,)
cong_nghe_man_hinh      (cong_nghe_man_hinh,)
cpu                                    (cpu,)
ram                                    (ram,)
bo_nho_trong                  (bo_nho_trong,)
pin                                    (pin,)
sim                                    (sim,)
dtype: object